# Urban Air Pollution Challenge


__Can you predict air quality in cities around the world using satellite data?__

For this challenge we’ll be digging deeper into Air quality data in several African cities, finding ways to track air quality and how it is changing, even in places without ground-based sensors. The collected weather data and daily observations are from the _Sentinel 5P satellite_ tracking various pollutants in the atmosphere. 

Our goal is to use the information from such data to predict _PM2.5_ particulate matter concentration (a common measure of air quality that normally requires ground-based sensors to measure) every day for each city. The data covers the last three months, spanning hundreds of cities across the globe.

_This is a Zindi data challenge, for more details check: [Urban Air Pollution challenge](https://zindi.africa/competitions/zindiweekendz-learning-urban-air-pollution-challenge)_


## Upload the data

In [ ]:
import sys
print(sys.executable)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Upload Train data
df_train=pd.read_csv('data/Train.csv')

# Upload Test data
df_test=pd.read_csv('data/Test.csv')

In [ ]:
df_train.head()

In [ ]:
# Y = df[]

## Data cleaning and feature engineering

In [ ]:
df_train.info()

In [ ]:
# X = 

## Splitting data for testing 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [ ]:
X_train.info()

In [ ]:
# fillna with mean.. 
# X_train[""] = X_train[""].fillna()

## Trainining the model

In [ ]:
## in order to exemplify how the predict will work.. we will save the y_train
X_test.to_csv("data/X_test.csv")
y_test.to_csv("data/y_test.csv")

In [ ]:
#training the model
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
y_train_pred = reg.predict(X_train)
mse = mean_squared_error(y_train, y_train_pred)
print(mse)